In [94]:
import math
import random
import os.path as osp


import os
import pandas as pd

from torch import Tensor

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

from torch.utils.data import Dataset as BaseDataset
from torch.utils.data import DataLoader

from tqdm import tqdm
from datetime import datetime

from sklearn.model_selection import train_test_split

In [95]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dilation=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                               kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU(inplace=True)

        # one kernel make one feature map
        # kernel 厚度 = channel 數
        self.is_changed = in_channels != out_channels
        self.trans = nn.Conv2d(in_channels, out_channels,
                               kernel_size=1, stride=stride)

    def forward(self, x):
        f_x = self.conv1(x)
        f_x = self.bn1(f_x)
        f_x = self.relu(f_x)
        f_x = self.conv2(f_x)
        f_x = self.bn2(f_x)

        if self.is_changed:
            x = self.trans(x)

        x = f_x + x
        x = self.relu(x)
        return x


class BottleNeck(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1, dilation=4):
        super(BottleNeck, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, padding=0)
        self.conv2 = nn.Conv2d(
            in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(
            in_channels=out_channels, out_channels=dilation*out_channels, kernel_size=1, padding=0)

        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        self.bn3 = nn.BatchNorm2d(num_features=out_channels * dilation)

        self.relu = nn.ReLU(inplace=True)

        # one kernel make one feature map
        # kernel 厚度 = channel 數
        self.is_changed = in_channels != (out_channels*dilation)
        self.trans = nn.Conv2d(in_channels, out_channels *
                               dilation, kernel_size=1, stride=stride)

    def forward(self, x):

        f_x = self.conv1(x)
        f_x = self.bn1(f_x)
        f_x = self.relu(f_x)
        f_x = self.conv2(f_x)
        f_x = self.bn2(f_x)
        f_x = self.relu(f_x)
        f_x = self.conv3(f_x)
        f_x = self.bn3(f_x)

        if self.is_changed:
            x = self.trans(x)

        x = f_x + x
        x = self.relu(x)
        return x


class _ResNet(nn.Module):
    def __init__(self, block, block_cnts, dilation=1):
        super(_ResNet, self).__init__()

        self.in_channels = 64
        self.out_channels = 64

        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=self.in_channels, kernel_size=7, stride=2, padding=3)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._layer(
            block, block_cnts[0], dilation, self.in_channels, self.out_channels, stride=1)
        self.layer2 = self._layer(
            block, block_cnts[1], dilation, self.in_channels, self.out_channels, stride=2)
        self.layer3 = self._layer(
            block, block_cnts[2], dilation, self.in_channels, self.out_channels, stride=2)
        self.layer4 = self._layer(
            block, block_cnts[3], dilation, self.in_channels, self.out_channels, stride=2)

        self.avg = nn.AdaptiveAvgPool2d(output_size=(1, 1))

        self.dense = nn.Linear(in_features=self.in_channels, out_features=1)
        self.activation = nn.Sigmoid()
        self.flatten = nn.Flatten(start_dim=1)

    def _layer(self, block, block_cnt, dilation, in_channels, out_channels, stride):
        # in_channels: param of previous block output channel
        # out_channels: param of current block input channel

        blocks = []
        blocks.append(
            block(in_channels=in_channels,
                  out_channels=out_channels, stride=stride)
        )

        for cnt in range(1, block_cnt):
            b = block(in_channels=dilation * out_channels,
                      out_channels=out_channels)
            blocks.append(b)

        self.in_channels = out_channels * dilation
        self.out_channels = out_channels * 2

        return nn.Sequential(*blocks)

    def forward(self, x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg(x)

        x = self.flatten(x)
        x = self.dense(x)

        x = self.activation(x)
        return x

In [112]:
root = os.getcwd()

dataset = "dataset1"
date_time = datetime.strftime(datetime.now(), "%Y-%m-%d_%H-%M")

### dataset 1
# num_ent = 2708
# feat_len = 1433
# embed_dim = 1600

### dataset 2
# num_ent = 3312
# embed_dim = 4000

### dataset 2
# num_ent = 877
# feat_len = 1703
# embed_dim = 1800

In [97]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [98]:
class Args:
    def __init__(self,
                num_ent=2708,
                feat_len=1703,
                embed_dim=1800,
                inp_drop=0.5,
                feat_drop=0.5,
                hid_drop=0.2,
                perm=1,
                k_w=40,
                k_h=40,
                jk_h=50,
                num_filt=96,
                ker_sz=3):
        self.num_ent = num_ent
        self.feat_len = feat_len
        self.embed_dim = embed_dim
        self.inp_drop = inp_drop
        self.feat_drop = feat_drop
        self.hid_drop = hid_drop
        self.perm = perm
        self.k_w = k_w
        self.k_h = k_h
        self.num_filt = num_filt
        self.ker_sz = ker_sz

In [99]:
train_df = pd.read_csv(os.path.join(root, "hw2_data", dataset, "raw", "train.csv"))

tmp = pd.DataFrame(data={
                    "from": train_df["to"], 
                    "to": train_df["from"],
                    "label": train_df["label"],
                    "id": train_df["id"]})

train_df = pd.concat([train_df, tmp], ignore_index=True)
train_df = train_df.drop_duplicates()

In [100]:
train_df, valid_df = train_test_split(train_df)

In [101]:
content_df = pd.read_csv(os.path.join(root, "hw2_data", dataset, "raw", "content.csv"), 
                         delimiter="\t", 
                         header=None)
content_df = content_df.sort_values(by=[0]).loc[:, 1:]

In [102]:
test_df = pd.read_csv(os.path.join(root, "hw2_data", dataset, "raw", "test.csv"))

In [103]:
class Dataset(BaseDataset):
    
    def __init__(self, dataframe, content, split="train"):
        
        # Read node features
        content = content.to_numpy()
        self.features = content
        self.dataframe = dataframe
        
        self.split = split
        
    def __len__(self):
        return len(self.dataframe)
        
    def __getitem__(self, idx):
        
        row = self.dataframe.iloc[idx]

        follower, followee = row["from"], row["to"]

        follower_feat = self.features[follower]
        followee_feat = self.features[followee]

        if self.split == "test":
            return follower, followee, follower_feat, followee_feat 
        
        return follower, followee, follower_feat, followee_feat, row["label"]

In [104]:
trainset = Dataset(train_df, content_df, split="train")
validset = Dataset(valid_df, content_df, split="valid")
testset = Dataset(test_df, content_df, split="test")

In [105]:
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
validloader = DataLoader(validset, batch_size=32, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

In [106]:
args = Args(num_ent=len(content_df), feat_len=len(content_df.iloc[0]))

In [107]:
args.feat_len

1433

In [108]:
def get_chequer_perm(perm=1, k_w=30, k_h=50):
    """
    Function to generate the chequer permutation required for InteractE model
    Parameters
    ----------

    Returns
    -------

    """
    embed_dim = k_w * k_h
#     ent_perm  = np.int32([np.random.permutation(embed_dim) for _ in range(perm)])
#     print(ent_perm)
    ent_perm  = np.int32([np.arange(embed_dim) for _ in range(perm)])
#     rel_perm  = np.int32([np.random.permutation(embed_dim) for _ in range(perm)])
#     print(rel_perm)
    rel_perm  = np.int32([np.arange(embed_dim) for _ in range(perm)])

    comb_idx = []
    for k in range(perm):
        temp = []
        ent_idx, rel_idx = 0, 0

        for i in range(embed_dim):
            if k % 2 == 0:
                if i % 2 == 0:
                    temp.append(ent_perm[k, ent_idx]); ent_idx += 1;
                    temp.append(rel_perm[k, rel_idx]+embed_dim); rel_idx += 1;
                else:
                    temp.append(rel_perm[k, rel_idx]+embed_dim); rel_idx += 1;
                    temp.append(ent_perm[k, ent_idx]); ent_idx += 1;
            else:
                if i % 2 == 0:
                    temp.append(rel_perm[k, rel_idx]+embed_dim); rel_idx += 1;
                    temp.append(ent_perm[k, ent_idx]); ent_idx += 1;
                else:
                    temp.append(ent_perm[k, ent_idx]); ent_idx += 1;
                    temp.append(rel_perm[k, rel_idx]+embed_dim); rel_idx += 1;

        comb_idx.append(temp)

    chequer_perm = torch.LongTensor(np.int32(comb_idx))
    return chequer_perm


In [109]:
chequer = get_chequer_perm(perm=args.perm, k_w=args.k_w, k_h=args.k_h)
chequer

tensor([[   0, 1600, 1601,  ..., 3198, 3199, 1599]])

In [110]:
class InteractE(torch.nn.Module):
    """
    Proposed method in the paper. Refer Section 6 of the paper for mode details 
    Parameters
    ----------
    params:         Hyperparameters of the model
    chequer_perm:   Reshaping to be used by the model

    Returns
    -------
    The InteractE model instance

    """
    def __init__(self, params, chequer_perm):
        super(InteractE, self).__init__()

        self.p = params
        self.ent_embed = torch.nn.Embedding(self.p.num_ent, self.p.embed_dim-self.p.feat_len, padding_idx=None); # xavier_normal_(self.ent_embed.weight)
        print(self.ent_embed)
        self.chequer_perm = chequer_perm


        self.resnet = _ResNet(BasicBlock, [2, 2, 2, 2], 1)
    
    def circular_padding_chw(self, batch, padding):
        upper_pad = batch[..., -padding:, :]
        lower_pad = batch[..., :padding, :]
        temp = torch.cat([upper_pad, batch, lower_pad], dim=2)

        left_pad = temp[..., -padding:]
        right_pad = temp[..., :padding]
        padded = torch.cat([left_pad, temp, right_pad], dim=3)
        return padded

    def forward(self, follower, followee, follower_id, followee_id):

#         print(follower_id)
        follower_emb = self.ent_embed(follower_id)
        if isinstance(follower, np.ndarray):
            follower = torch.from_numpy(follower)
        follower_emb = torch.cat((follower, follower_emb), dim=-1)
  
        
        followee_emb = self.ent_embed(followee_id)
        if isinstance(followee, np.ndarray):
            followee = torch.from_numpy(followee)
        followee_emb = torch.cat((followee, followee_emb), dim=-1)
        
        comb_emb = torch.cat([follower_emb, followee_emb], dim=-1)
        chequer_perm = comb_emb[:, self.chequer_perm] # batch, 1, embed_size

        stack_inp = chequer_perm.reshape((-1, self.p.perm, 2*self.p.k_w, self.p.k_h)) # batch, 1, 2*k_w, k_h
        
        #### start
        
#         stack_inp = self.bn0(stack_inp)
        
#         stack_inp = self.inp_drop(stack_inp)
        x = self.circular_padding_chw(stack_inp, self.p.ker_sz//2)
        
        pred = self.resnet(x)

        return pred


In [111]:
def load_checkpoint(filepath, device):
    
    model = InteractE(args, chequer).to(device)

    if os.path.exists(filepath):
        print("pretrained finded")
        checkpoint = torch.load(filepath)
        model.load_state_dict(checkpoint['model_stat'])
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        optimizer.load_state_dict(checkpoint['optimizer_stat'])

    else:
        print("use a new optimizer")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    return model, optimizer

In [115]:
model, optimizer = load_checkpoint(os.path.join(root, "results", dataset, "conv", 
                                                "{}_{}.pth".format(date_time, dataset)), 
                                   device=device)

Embedding(2708, 367)
use a new optimizer


In [116]:
def train(model, optim, loader, device):
    
    model.train()
    model = model.to(device)
    
    total_loss = 0
    for index, data in tqdm(enumerate(loader)):
        
        optim.zero_grad()
        
        follower, followee, follower_feat, followee_feat, label = data
        
        follower = follower.to(device)
        followee = followee.to(device)
        follower_feat = follower_feat.to(device)
        followee_feat = followee_feat.to(device)
        label = label.to(device).float()

        pred = model(follower_feat, followee_feat, follower, followee) 
        pred = torch.squeeze(pred)
        
        loss = torch.nn.BCELoss()(pred, label)

        loss.backward()
        optim.step()
        
        total_loss += loss.item()

    total_loss = total_loss/(index+1)
    return total_loss

In [117]:
@torch.no_grad()
def test(model, loader, device="cpu"):
    
    from sklearn.metrics import roc_auc_score
    
    model.eval()
    model = model.to(device)
    
    y_pred, y_true = [], []
    for data in tqdm(loader):
        follower, followee, follower_feat, followee_feat, label = data
        
        follower = follower.to(device)
        followee = followee.to(device)
        follower_feat = follower_feat.to(device)
        followee_feat = followee_feat.to(device)
        label = label.to(device).float()
        
        pred = model(follower_feat, followee_feat, follower, followee) 
        pred = torch.squeeze(pred)
        
        y_pred.append(pred.view(-1).cpu())
        y_true.append(label.view(-1).cpu())
     
    return roc_auc_score(torch.cat(y_true), torch.cat(y_pred))

In [118]:
@torch.no_grad()
def predict(model, loader, device="cpu"):
    
#     from sklearn.metrics import roc_auc_score
    
    model.eval()
    model = model.to(device)
    
    y_pred = []
    for data in tqdm(loader):
        follower, followee, follower_feat, followee_feat = data
        
        follower = follower.to(device)
        followee = followee.to(device)
        follower_feat = follower_feat.to(device)
        followee_feat = followee_feat.to(device)
        
        pred = model(follower_feat, followee_feat, follower, followee) 
        pred = torch.squeeze(pred)
        
        y_pred.append(pred.view(-1).cpu())
     
    return torch.cat(y_pred)

In [120]:
valid_auc = 0
for epoch in range(100):

    loss = train(model, optimizer, trainloader, device)
    train_auc = test(model, trainloader, device)
    auc = test(model, validloader, device)
   
    if auc > valid_auc:
        valid_auc = auc
        
        checkpoint = {
            'model_stat': model.state_dict(),
            'optimizer_stat': optimizer.state_dict(),
        }
        
        torch.save(checkpoint, os.path.join(root, "results", dataset, "conv", 
                                                "{}_{}.pth".format(date_time, dataset)))
        print("\nSave Model")
    
    print("Epoch: {}, loss={:.5f}, valid_auc={:.5f}, train_auc={:.5f}".format(epoch+1, loss, auc, train_auc))
    

408it [00:09, 43.21it/s]
100%|██████████| 136/136 [00:01<00:00, 127.72it/s]



Save Model
Epoch: 1, loss=0.69190, valid_auc=0.58356, train_auc=0.68590


408it [00:09, 43.85it/s]
100%|██████████| 136/136 [00:01<00:00, 125.13it/s]



Save Model
Epoch: 2, loss=0.66286, valid_auc=0.59367, train_auc=0.71047


408it [00:09, 44.28it/s]
100%|██████████| 136/136 [00:01<00:00, 128.49it/s]


...

Epoch: 98, loss=0.01226, valid_auc=0.72993, train_auc=0.99995



408it [00:09, 42.24it/s]
100%|██████████| 136/136 [00:01<00:00, 129.87it/s]


Epoch: 99, loss=0.01847, valid_auc=0.72587, train_auc=0.99994


408it [00:09, 43.13it/s]
100%|██████████| 136/136 [00:00<00:00, 138.23it/s]

Epoch: 100, loss=0.01311, valid_auc=0.72610, train_auc=0.99966


In [89]:
model, optimizer = load_checkpoint(os.path.join(root, "results", dataset, "conv", "weight.pth"), device=device)

Embedding(877, 97)
pretrained finded


In [90]:
pred = predict(model, testloader, device)

100%|██████████| 21/21 [00:00<00:00, 61.72it/s]


In [91]:
pred = pred.detach().cpu().numpy()
# pred = np.round(pred, 3)
pred

array([9.99296665e-01, 9.99974608e-01, 9.99904752e-01, 9.81246769e-01,
       9.41093604e-05, 3.11104268e-05, 9.99999166e-01, 2.24112961e-02,
       9.99894381e-01, 9.99646306e-01, 4.32964180e-06, 9.97670829e-01,
       2.91336537e-03, 9.98132527e-01, 4.62299140e-05, 9.99988914e-01,
       1.76502326e-07, 9.99999762e-01, 1.65335496e-03, 9.99194682e-01,
       1.00000000e+00, 9.99995828e-01, 9.99987245e-01, 3.67666507e-04,
       2.47902626e-05, 9.99567568e-01, 9.99998808e-01, 2.89711033e-06,
       9.99994516e-01, 9.99999881e-01, 9.98152077e-01, 9.99993324e-01,
       4.22160685e-01, 9.99997497e-01, 9.99997377e-01, 9.99977469e-01,
       1.86935576e-05, 9.99999762e-01, 9.99999881e-01, 9.99915004e-01,
       9.99989629e-01, 9.99951839e-01, 9.97748911e-01, 8.24332631e-07,
       7.14886427e-01, 5.42289818e-05, 9.99966025e-01, 3.76991329e-06,
       9.99999642e-01, 2.09501650e-05, 9.99803841e-01, 5.23268938e-01,
       3.84405375e-01, 3.82165535e-06, 9.99978065e-01, 8.77665798e-07,
      

In [92]:
test_df["prob"] = pred
test_df.to_csv("conv_all.csv", index=False)

In [93]:
upload = test_df[["id", "prob"]]
upload.to_csv("upload.csv", index=False)